In [1]:
#
# Data Below is sourced from Kaggle.com, specifically at this URL:
# https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews
# 3 Files:
#   beers.csv
#   breweries.csv
#   reviews.csv
#
# reviews.csv is rather large, so we will prune it down to what we need
#
import pandas as pd
import numpy as np
import zipfile
import os 

In [2]:
# This is a large file, will need to prune it
df = pd.read_csv("../Resources/reviews.csv")

In [3]:
# Get the column names to see what is here
df.columns

Index(['beer_id', 'username', 'date', 'text', 'look', 'smell', 'taste', 'feel',
       'overall', 'score'],
      dtype='object')

In [4]:
# Column types to ensure proper analysis
print("types of each columns: \n\n",df.dtypes)
print("\ninformation of the columns: \n")
print(df.info())

types of each columns: 

 beer_id       int64
username     object
date         object
text         object
look        float64
smell       float64
taste       float64
feel        float64
overall     float64
score       float64
dtype: object

information of the columns: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9073128 entries, 0 to 9073127
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   beer_id   int64  
 1   username  object 
 2   date      object 
 3   text      object 
 4   look      float64
 5   smell     float64
 6   taste     float64
 7   feel      float64
 8   overall   float64
 9   score     float64
dtypes: float64(6), int64(1), object(3)
memory usage: 692.2+ MB
None


In [5]:
df['text'].head()

0       750 ml bottle, 2016 vintage, bottle #304 of...
1                                                     
2                                                     
3       0% 16 oz can. Funny story: As I finally wal...
4       Classic TH NEIPA. Overflowing head and bouq...
Name: text, dtype: object

In [6]:
df.loc[4,'text']

'\xa0\xa0 Classic TH NEIPA. Overflowing head and bouquet. Strong flavor but smoother than 8.2%. Maybe peach nose but overall a general citrus punch.'

In [7]:
df.loc[1,'text']

'\xa0\xa0'

In [8]:
df[df['text']=='\xa0\xa0' ].count()

beer_id     6085135
username    6085107
date        6085135
text        6085135
look        2459358
smell       2459358
taste       2459358
feel        2459358
overall     2459358
score       6085135
dtype: int64

In [9]:
# None of the text is '' but has funny characters in it when it should be empty
# df['text'].replace('\xa0\xa0', np.nan, inplace=True)

In [10]:
#print("Overview of missing values in the dataset: \n",df.isnull().sum())

In [11]:
# Drop the na values
# df=df.dropna()
# print("After dropping the missing values: \n",df.info())

In [12]:
# Check to see if any left
# df.isnull().sum()

In [13]:
# Let's check for duplication amongst reviews
df[df.duplicated(['beer_id', 'username'])]

,beer_id,username,date,text,look,smell,taste,feel,overall,score
14692,47658,NaN,2012-09-19,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,5.00
16925,47658,NaN,2011-12-01,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,5.00
82457,48824,NaN,2012-09-09,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,5.00
86388,48873,NaN,2012-05-28,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,3.75
102911,50443,NaN,2012-09-11,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,3.25
...,...,...,...,...,...,...,...,...,...,...
9037311,86149,NaN,2012-12-27,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,3.75
9041229,86149,NaN,2012-11-08,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,4.25
9047181,57285,NaN,2012-12-23,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,1.50
9048483,57285,NaN,2012-09-30,You had this beer. Write a review.,NaN,NaN,NaN,NaN,NaN,2.75


In [14]:
df[df['username'].isnull()].count()

beer_id     3815
username       0
date        3815
text        3815
look        1134
smell       1134
taste       1134
feel        1134
overall     1134
score       3815
dtype: int64

In [15]:
df2 = df[~df['look'].isnull()]

In [16]:
# Quick check of metrics
round(df2.describe(),2)

,beer_id,look,smell,taste,feel,overall,score
count,5283110.00,5283110.00,5283110.00,5283110.00,5283110.00,5283110.00,5283110.00
mean,91498.29,3.95,3.89,3.92,3.88,3.92,3.91
std,88956.61,0.55,0.61,0.63,0.60,0.61,0.57
min,3.00,1.00,1.00,1.00,1.00,1.00,1.00
25%,15813.00,3.75,3.50,3.50,3.50,3.50,3.65
50%,64184.00,4.00,4.00,4.00,4.00,4.00,4.00
75%,146497.00,4.25,4.25,4.25,4.25,4.25,4.27
max,373104.00,5.00,5.00,5.00,5.00,5.00,5.00


In [17]:
len(df2)

5283110

In [18]:
df2[df2.duplicated(['beer_id', 'username'])]

,beer_id,username,date,text,look,smell,taste,feel,overall,score
115163,50697,NaN,2011-08-01,a-golden hue with a two finger head and liv...,4.0,4.5,4.5,4.0,4.5,4.42
627794,34094,NaN,2011-07-22,2011 Bottle a-jet black with a tan head. s-...,4.0,3.0,3.5,3.0,3.0,3.26
862262,33,NaN,2011-05-12,0% Appearance - I love the name of the beer...,4.0,4.0,4.5,4.0,4.0,4.20
884572,39,NaN,2006-12-19,"12oz. bottle. Hazy amber fluid, with a medi...",3.5,4.0,4.0,4.0,4.5,4.07
1019270,652,NaN,2011-12-15,Appearance - ruby-tinted dark brown in colo...,4.0,4.5,4.0,4.5,4.0,4.17
...,...,...,...,...,...,...,...,...,...,...
7406284,40057,NaN,2008-02-25,"22oz. bottle. Slightly hazy, light gold in ...",4.0,4.0,4.5,4.5,4.5,4.35
7521376,33467,NaN,2006-11-06,"22oz. bomber. Hazy, straw yellow to gold. V...",3.5,3.5,4.0,4.0,4.0,3.85
7588904,33832,NaN,2008-06-21,"12 oz. bottle Poured a very dark brown, wit...",4.0,4.0,4.0,4.0,4.0,4.00
7598375,38586,NaN,2011-05-27,Appearance - honey-coloured with a long las...,3.0,3.5,3.5,3.5,3.5,3.47


In [19]:
df2 =df2[df2['text']!='\xa0\xa0' ]

In [20]:
df2.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03
3,125646,GratefulBeerGuy,2017-12-20,0% 16 oz can. Funny story: As I finally wal...,4.75,4.75,4.50,4.50,4.50,4.58
4,125646,LukeGude,2017-12-20,Classic TH NEIPA. Overflowing head and bouq...,4.25,4.50,4.25,4.25,4.25,4.31
7,125646,MFMB,2017-12-16,Pours a creamy opaque light straw yellow wi...,4.75,4.50,4.50,4.50,4.50,4.52
13,125646,jngrizzaffi,2017-12-10,Pours a cloudy yellow color with a thin foa...,4.50,4.50,4.50,4.75,4.50,4.53


In [21]:
len(df2)

2823752

In [22]:
df2.describe()

,beer_id,look,smell,taste,feel,overall,score
count,2.823752e+06,2.823752e+06,2.823752e+06,2.823752e+06,2.823752e+06,2.823752e+06,2.823752e+06
mean,6.036305e+04,3.899786e+00,3.812785e+00,3.852731e+00,3.816532e+00,3.860212e+00,3.844702e+00
std,7.385312e+04,5.892651e-01,6.628718e-01,6.958242e-01,6.529130e-01,6.707193e-01,6.058625e-01
min,3.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,4.202000e+03,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.560000e+00
50%,3.769100e+04,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,3.950000e+00
75%,7.953300e+04,4.250000e+00,4.250000e+00,4.500000e+00,4.250000e+00,4.250000e+00,4.240000e+00
max,3.729140e+05,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


In [23]:
# Read in the beer data
beer_df = pd.read_csv('../Resources/beers.csv')
beer_df.head()

,id,name,brewery_id,state,country,style,availability,abv,notes,retired
0,202522,Olde Cogitator,2199,CA,US,English Oatmeal Stout,Rotating,7.3,No notes at this time.,f
1,82352,Konrads Stout Russian Imperial Stout,18604,NaN,NO,Russian Imperial Stout,Rotating,10.4,No notes at this time.,f
2,214879,Scottish Right,44306,IN,US,Scottish Ale,Year-round,4.0,No notes at this time.,t
3,320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,8.7,Every time this year,f
4,246438,Peaches-N-Cream,44617,PA,US,American Cream Ale,Rotating,5.1,No notes at this time.,f


In [24]:
len(beer_df)

358873

In [25]:
# As we said review data is quite large, let's restrict this to US market
us_beer_df = beer_df[beer_df['country']== 'US']
us_beer_df.head()

,id,name,brewery_id,state,country,style,availability,abv,notes,retired
0,202522,Olde Cogitator,2199,CA,US,English Oatmeal Stout,Rotating,7.3,No notes at this time.,f
2,214879,Scottish Right,44306,IN,US,Scottish Ale,Year-round,4.0,No notes at this time.,t
3,320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,8.7,Every time this year,f
4,246438,Peaches-N-Cream,44617,PA,US,American Cream Ale,Rotating,5.1,No notes at this time.,f
6,108605,Icon Sender,22598,CA,US,American Lager,Year-round,5.6,No notes at this time.,f


In [26]:
# Merge the beer and review data
beer_review_merged_df = pd.merge(df2, us_beer_df, how="inner", left_on="beer_id", right_on="id")
len(beer_review_merged_df)

2190820

In [27]:
beer_review_merged_df.columns

Index(['beer_id', 'username', 'date', 'text', 'look', 'smell', 'taste', 'feel',
       'overall', 'score', 'id', 'name', 'brewery_id', 'state', 'country',
       'style', 'availability', 'abv', 'notes', 'retired'],
      dtype='object')

In [28]:
# It is now time to trim the data we no longer want (text)
trimmed_df = beer_review_merged_df[['beer_id', 'username', 'date', 'text', 'look', 'smell', 'taste', 'feel',
       'overall', 'score', 'name', 'brewery_id', 'state','style', 'availability', 'abv']]

In [29]:
# Read the brewery data
brewery_df = pd.read_csv('../Resources/breweries.csv')
brewery_df.head()

,id,name,city,state,country,notes,types
0,19730,Brouwerij Danny,Erpe-Mere,NaN,BE,No notes at this time.,Brewery
1,32541,Coachella Valley Brewing Co,Thousand Palms,CA,US,No notes at this time.,"Brewery, Bar, Beer-to-go"
2,44736,Beef 'O' Brady's,Plant City,FL,US,No notes at this time.,"Bar, Eatery"
3,23372,Broadway Wine Merchant,Oklahoma City,OK,US,No notes at this time.,Store
4,35328,Brighton Beer Dispensary (DUPLICATE),Brighton,GB2,GB,Duplicate of https://www.beeradvocate.com/beer...,"Bar, Eatery"


In [30]:
# Trim down to US breweries only

us_brewery_df = brewery_df[brewery_df['country']== 'US']
us_brewery_df.head()

,id,name,city,state,country,notes,types
1,32541,Coachella Valley Brewing Co,Thousand Palms,CA,US,No notes at this time.,"Brewery, Bar, Beer-to-go"
2,44736,Beef 'O' Brady's,Plant City,FL,US,No notes at this time.,"Bar, Eatery"
3,23372,Broadway Wine Merchant,Oklahoma City,OK,US,No notes at this time.,Store
5,31561,Teddy's Tavern,Seattle,WA,US,No notes at this time.,"Bar, Beer-to-go"
9,41278,The Other End,Destin,FL,US,No notes at this time.,"Bar, Eatery"


In [31]:
# Now lets merge all three datasets togther
brewery_beer_review_merged_df = pd.merge(trimmed_df, us_brewery_df, how="inner", left_on="brewery_id", right_on="id")
len(brewery_beer_review_merged_df)

2190820

In [32]:
brewery_beer_review_merged_df.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score,...,style,availability,abv,id,name_y,city,state_y,country,notes,types
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03,...,American Imperial Stout,Limited (brewed once),10.8,28094,Four String Brewing Company,Columbus,OH,US,No notes at this time.,"Brewery, Bar"
1,184647,Try-em-all,2017-09-27,"Clear gold in color, light head, very littl...",4.00,4.00,3.50,3.50,3.75,3.70,...,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,US,No notes at this time.,"Brewery, Bar"
2,184647,ScorpioBeerLover,2017-03-22,Reminds me of pretty much every Czech Pilsn...,4.00,4.00,4.25,3.75,4.00,4.08,...,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,US,No notes at this time.,"Brewery, Bar"
3,184647,beergoot,2016-10-08,Pale yellow body; wispy white head. Very fa...,3.25,3.25,3.50,3.50,3.50,3.43,...,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,US,No notes at this time.,"Brewery, Bar"
4,184647,woodychandler,2016-09-25,The CANQuest (tm) is honored to add anythin...,3.75,3.75,3.50,3.50,3.50,3.58,...,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,US,No notes at this time.,"Brewery, Bar"


In [33]:
brewery_beer_review_merged_df.columns

Index(['beer_id', 'username', 'date', 'text', 'look', 'smell', 'taste', 'feel',
       'overall', 'score', 'name_x', 'brewery_id', 'state_x', 'style',
       'availability', 'abv', 'id', 'name_y', 'city', 'state_y', 'country',
       'notes', 'types'],
      dtype='object')

In [34]:
# Rename some key columns for better data definition
brewery_beer_review_merged_df = brewery_beer_review_merged_df.rename(columns={'name_y':'brewery_name', 'name_x':'beer_name','state_x':'review_state', 'state_y':'brewery_state'})

In [35]:
brewery_beer_review_merged_df.columns

Index(['beer_id', 'username', 'date', 'text', 'look', 'smell', 'taste', 'feel',
       'overall', 'score', 'beer_name', 'brewery_id', 'review_state', 'style',
       'availability', 'abv', 'id', 'brewery_name', 'city', 'brewery_state',
       'country', 'notes', 'types'],
      dtype='object')

In [36]:
# Drop unwanted columns
brewery_beer_review_merged_df = brewery_beer_review_merged_df.drop(['brewery_id', 'notes', 'country'], axis=1)

In [37]:
brewery_beer_review_merged_df.columns

Index(['beer_id', 'username', 'date', 'text', 'look', 'smell', 'taste', 'feel',
       'overall', 'score', 'beer_name', 'review_state', 'style',
       'availability', 'abv', 'id', 'brewery_name', 'city', 'brewery_state',
       'types'],
      dtype='object')

In [38]:
# Now that we dropped some columns let rename to be more descriptive
brewery_beer_review_merged_df = brewery_beer_review_merged_df.rename(columns={'id':'brewery_id', 'city':'brewery_city', 'types':'brewery_types'})

In [39]:
brewery_beer_review_merged_df.columns

Index(['beer_id', 'username', 'date', 'text', 'look', 'smell', 'taste', 'feel',
       'overall', 'score', 'beer_name', 'review_state', 'style',
       'availability', 'abv', 'brewery_id', 'brewery_name', 'brewery_city',
       'brewery_state', 'brewery_types'],
      dtype='object')

In [40]:
brewery_beer_review_merged_df.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score,beer_name,review_state,style,availability,abv,brewery_id,brewery_name,brewery_city,brewery_state,brewery_types
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03,Motorbreath Imperial Stout,OH,American Imperial Stout,Limited (brewed once),10.8,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
1,184647,Try-em-all,2017-09-27,"Clear gold in color, light head, very littl...",4.00,4.00,3.50,3.50,3.75,3.70,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
2,184647,ScorpioBeerLover,2017-03-22,Reminds me of pretty much every Czech Pilsn...,4.00,4.00,4.25,3.75,4.00,4.08,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
3,184647,beergoot,2016-10-08,Pale yellow body; wispy white head. Very fa...,3.25,3.25,3.50,3.50,3.50,3.43,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
4,184647,woodychandler,2016-09-25,The CANQuest (tm) is honored to add anythin...,3.75,3.75,3.50,3.50,3.50,3.58,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"


In [41]:
# Do an additional NULL check now that we merged
brewery_beer_review_merged_df.isnull().sum()

beer_id              0
username           810
date                 0
text                 0
look                 0
smell                0
taste                0
feel                 0
overall              0
score                0
beer_name            0
review_state     12234
style                0
availability         0
abv              51282
brewery_id           0
brewery_name         0
brewery_city     12234
brewery_state    12234
brewery_types        0
dtype: int64

In [42]:
# Nulls don't look significant enough let's drop them
brewery_beer_review_merged_df = brewery_beer_review_merged_df.dropna()

In [43]:
# Create the file paths
csv_file_path = os.path.join('../Resources',"reviews_beer_brewery.csv")
zip_file_path = os.path.join('../Resources',"reviews_beer_brewery.zip")

In [44]:
# Save uncompressed dataframe to zip
brewery_beer_review_merged_df.to_csv(csv_file_path, index=False)

In [45]:
# Compress the final output
zout = zipfile.ZipFile(zip_file_path, "w", zipfile.ZIP_DEFLATED)
zout.write(csv_file_path)
zout.close()

In [46]:
# Make sure we can read the zip file just created
zip_df = pd.read_csv(zip_file_path)
zip_df.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score,beer_name,review_state,style,availability,abv,brewery_id,brewery_name,brewery_city,brewery_state,brewery_types
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03,Motorbreath Imperial Stout,OH,American Imperial Stout,Limited (brewed once),10.8,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
1,184647,Try-em-all,2017-09-27,"Clear gold in color, light head, very littl...",4.00,4.00,3.50,3.50,3.75,3.70,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
2,184647,ScorpioBeerLover,2017-03-22,Reminds me of pretty much every Czech Pilsn...,4.00,4.00,4.25,3.75,4.00,4.08,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
3,184647,beergoot,2016-10-08,Pale yellow body; wispy white head. Very fa...,3.25,3.25,3.50,3.50,3.50,3.43,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"
4,184647,woodychandler,2016-09-25,The CANQuest (tm) is honored to add anythin...,3.75,3.75,3.50,3.50,3.50,3.58,Payback Pilsner,OH,Bohemian Pilsener,Rotating,5.1,28094,Four String Brewing Company,Columbus,OH,"Brewery, Bar"


In [47]:
zip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127677 entries, 0 to 2127676
Data columns (total 20 columns):
 #   Column         Dtype  
---  ------         -----  
 0   beer_id        int64  
 1   username       object 
 2   date           object 
 3   text           object 
 4   look           float64
 5   smell          float64
 6   taste          float64
 7   feel           float64
 8   overall        float64
 9   score          float64
 10  beer_name      object 
 11  review_state   object 
 12  style          object 
 13  availability   object 
 14  abv            float64
 15  brewery_id     int64  
 16  brewery_name   object 
 17  brewery_city   object 
 18  brewery_state  object 
 19  brewery_types  object 
dtypes: float64(7), int64(2), object(11)
memory usage: 324.7+ MB
